# Microsoft SharePoint | Microsoft SharePoint

> [Microsoft SharePoint](https://en.wikipedia.org/wiki/SharePoint)は、ワークフローアプリケーション、"リスト"データベース、その他のウェブパーツやセキュリティ機能を使用して、ビジネスチームが共同で作業することを支援するウェブサイトベースのコラボレーションシステムで、Microsoftによって開発されました。
>
> > [Microsoft SharePoint](https://en.wikipedia.org/wiki/SharePoint) is a website-based collaboration system that uses workflow applications, “list” databases, and other web parts and security features to empower business teams to work together developed by Microsoft.

このノートブックでは、[SharePoint Document Library](https://support.microsoft.com/en-us/office/what-is-a-document-library-3b5976dd-65cf-4c9e-bf5a-713c10ca2872)からドキュメントを読み込む方法について説明しています。現在、docx、doc、pdfファイルのみがサポートされています。

> This notebook covers how to load documents from the [SharePoint Document Library](https://support.microsoft.com/en-us/office/what-is-a-document-library-3b5976dd-65cf-4c9e-bf5a-713c10ca2872). Currently, only docx, doc, and pdf files are supported.

## Prerequisites | 前提条件

1. [Microsoft identity platform](https://learn.microsoft.com/en-us/azure/active-directory/develop/quickstart-register-app)の指示に従ってアプリケーションを登録してください。

   > Register an application with the [Microsoft identity platform](https://learn.microsoft.com/en-us/azure/active-directory/develop/quickstart-register-app) instructions.

2. 登録が完了すると、Azureポータルはアプリ登録の「概要」ペインを表示します。そこでアプリケーション（クライアント）IDを確認できます。`client ID`とも呼ばれるこの値は、Microsoft identity platform内であなたのアプリケーションを一意に識別するものです。

   > When registration finishes, the Azure portal displays the app registration's Overview pane. You see the Application (client) ID. Also called the `client ID`, this value uniquely identifies your application in the Microsoft identity platform.

3. **項目1**で進める手順の中で、リダイレクトURIを`https://login.microsoftonline.com/common/oauth2/nativeclient`に設定することができます。

   > During the steps you will be following at **item 1**, you can set the redirect URI as `https://login.microsoftonline.com/common/oauth2/nativeclient`

4. **項目1**での手順に従いながら、アプリケーションシークレットセクションで新しいパスワード（`client_secret`）を生成してください。

   > During the steps you will be following at **item 1**, generate a new password (`client_secret`) under Application Secrets section.

5. この[ドキュメント](https://learn.microsoft.com/en-us/azure/active-directory/develop/quickstart-configure-app-expose-web-apis#add-a-scope)の指示に従って、アプリケーションに以下の`SCOPES`（`offline_access`と`Sites.Read.All`）を追加してください。

   > Follow the instructions at this [document](https://learn.microsoft.com/en-us/azure/active-directory/develop/quickstart-configure-app-expose-web-apis#add-a-scope) to add the following `SCOPES` (`offline_access` and `Sites.Read.All`) to your application.

6. あなたの**ドキュメントライブラリ**からファイルを取得するには、そのIDが必要です。それを取得するためには、`Tenant Name`、`Collection ID`、そして`Subsite ID`の値が必要です。

   > To retrieve files from your **Document Library**, you will need its ID. To obtain it, you will need values of `Tenant Name`, `Collection ID`, and `Subsite ID`.

7. `Tenant Name`を見つけるには、この[ドキュメント](https://learn.microsoft.com/en-us/azure/active-directory-b2c/tenant-management-read-tenant-name)の指示に従ってください。これを取得したら、値から`.onmicrosoft.com`を削除し、残りの部分をあなたの`Tenant Name`として保持してください。

   > To find your `Tenant Name` follow the instructions at this [document](https://learn.microsoft.com/en-us/azure/active-directory-b2c/tenant-management-read-tenant-name). Once you got this, just remove `.onmicrosoft.com` from the value and hold the rest as your `Tenant Name`.

8. `Collection ID`と`Subsite ID`を取得するには、**SharePoint**の`site-name`が必要です。あなたの`SharePoint`サイトのURLは、`https://<tenant-name>.sharepoint.com/sites/<site-name>`という形式になっています。このURLの最後の部分が`site-name`です。

   > To obtain your `Collection ID` and `Subsite ID`, you will need your **SharePoint** `site-name`. Your `SharePoint` site URL has the following format `https://<tenant-name>.sharepoint.com/sites/<site-name>`. The last part of this URL is the `site-name`.

9. サイトの `Collection ID` を取得するには、ブラウザでこのURLにアクセスしてください：`https://<tenant>.sharepoint.com/sites/<site-name>/_api/site/id`。そして、`Edm.Guid` プロパティの値をコピーしてください。

   > To Get the Site `Collection ID`, hit this URL in the browser: `https://<tenant>.sharepoint.com/sites/<site-name>/_api/site/id` and copy the value of the `Edm.Guid` property.

10. `Subsite ID`（またはweb ID）を取得するには、`https://<tenant>.sharepoint.com/<site-name>/_api/web/id`にアクセスし、`Edm.Guid`プロパティの値をコピーしてください。

    > To get the `Subsite ID` (or web ID) use: `https://<tenant>.sharepoint.com/<site-name>/_api/web/id` and copy the value of the `Edm.Guid` property.

11. `SharePoint site ID`は次の形式を持っています：`<tenant-name>.sharepoint.com,<Collection ID>,<subsite ID>`。この値を保持して次のステップで使用できます。

    > The `SharePoint site ID` has the following format: `<tenant-name>.sharepoint.com,<Collection ID>,<subsite ID>`. You can hold that value to use in the next step.

12. `Document Library ID`を取得するには、[Graph Explorer Playground](https://developer.microsoft.com/en-us/graph/graph-explorer)にアクセスしてください。最初のステップは、**SharePoint**サイトに関連付けられたアカウントでログインしていることを確認することです。次に、`https://graph.microsoft.com/v1.0/sites/<SharePoint site ID>/drive`にリクエストを送り、応答として返されるペイロードには`id`フィールドが含まれており、その`id`があなたの`Document Library ID`を示しています。

    > Visit the [Graph Explorer Playground](https://developer.microsoft.com/en-us/graph/graph-explorer) to obtain your `Document Library ID`. The first step is to ensure you are logged in with the account associated with your **SharePoint** site. Then you need to make a request to `https://graph.microsoft.com/v1.0/sites/<SharePoint site ID>/drive` and the response will return a payload with a field `id` that holds the ID of your `Document Library ID`.


## 🧑 Instructions for ingesting your documents from SharePoint Document Library | 🧑 SharePoint Document Libraryからドキュメントを取り込むための指示

### 🔑 Authentication | 🔑 認証

デフォルトでは、`SharePointLoader`は`CLIENT_ID`と`CLIENT_SECRET`の値がそれぞれ`O365_CLIENT_ID`と`O365_CLIENT_SECRET`という名前の環境変数として保存されていることを期待しています。これらの環境変数は、アプリケーションのルートにある`.env`ファイルを通じて、またはスクリプトで以下のコマンドを使用して設定することができます。

> By default, the `SharePointLoader` expects that the values of `CLIENT_ID` and `CLIENT_SECRET` must be stored as environment variables named `O365_CLIENT_ID` and `O365_CLIENT_SECRET` respectively. You could pass those environment variables through a `.env` file at the root of your application or using the following command in your script.

```python
os.environ['O365_CLIENT_ID'] = "YOUR CLIENT ID"
os.environ['O365_CLIENT_SECRET'] = "YOUR CLIENT SECRET"
```

このローダーは、[*ユーザーに代わって*](https://learn.microsoft.com/en-us/graph/auth-v2-user?context=graph%2Fapi%2F1.0\&view=graph-rest-1.0)と呼ばれる認証方式を使用しています。これはユーザーの同意を必要とする2段階認証です。ローダーをインスタンス化すると、アプリに必要な権限への同意をユーザーに求めるURLが表示されます。ユーザーはこのURLを訪れてアプリケーションへの同意を与える必要があります。その後、ユーザーは結果として得られるページのURLをコピーしてコンソールに貼り付けます。メソッドはログイン試行が成功した場合にTrueを返します。

> This loader uses an authentication called [*on behalf of a user*](https://learn.microsoft.com/en-us/graph/auth-v2-user?context=graph%2Fapi%2F1.0\&view=graph-rest-1.0). It is a 2 step authentication with user consent. When you instantiate the loader, it will call will print a url that the user must visit to give consent to the app on the required permissions. The user must then visit this url and give consent to the application. Then the user must copy the resulting page url and paste it back on the console. The method will then return True if the login attempt was succesful.

```python
from langchain.document_loaders.sharepoint import SharePointLoader

loader = SharePointLoader(document_library_id="YOUR DOCUMENT LIBRARY ID")
```

認証が完了すると、ローダーはトークン（`o365_token.txt`）を`~/.credentials/`フォルダに保存します。このトークンは、以前説明したコピー＆ペーストの手順を踏まずに後で認証に使用できます。このトークンを認証に使用するためには、ローダーのインスタンスを作成する際に`auth_with_token`パラメータをTrueに変更する必要があります。

> Once the authentication has been done, the loader will store a token (`o365_token.txt`) at `~/.credentials/` folder. This token could be used later to authenticate without the copy/paste steps explained earlier. To use this token for authentication, you need to change the `auth_with_token` parameter to True in the instantiation of the loader.

```python
from langchain.document_loaders.sharepoint import SharePointLoader

loader = SharePointLoader(document_library_id="YOUR DOCUMENT LIBRARY ID", auth_with_token=True)
```

### 🗂️ Documents loader | 🗂️ ドキュメントローダー

#### 📑 Loading documents from a Document Library Directory | 📑 ドキュメントライブラリディレクトリからドキュメントを読み込む

`SharePointLoader`は、ドキュメントライブラリ内の特定のフォルダからドキュメントを読み込むことができます。たとえば、ドキュメントライブラリ内の`Documents/marketing`フォルダに格納されているすべてのドキュメントを読み込みたい場合に使用できます。

> `SharePointLoader` can load documents from a specific folder within your Document Library. For instance, you want to load all documents that are stored at `Documents/marketing` folder within your Document Library.

```python
from langchain.document_loaders.sharepoint import SharePointLoader

loader = SharePointLoader(document_library_id="YOUR DOCUMENT LIBRARY ID", folder_path="Documents/marketing", auth_with_token=True)
documents = loader.load()
```

#### 📑 Loading documents from a list of Documents IDs | 📑 ドキュメントIDのリストからドキュメントを読み込む

別の方法として、読み込みたい各ドキュメントの`object_id`のリストを提供することも可能です。そのためには、[Microsoft Graph API](https://developer.microsoft.com/en-us/graph/graph-explorer)にクエリを実行して、興味のあるすべてのドキュメントIDを見つけ出す必要があります。この[リンク](https://learn.microsoft.com/en-us/graph/api/resources/onedrive?view=graph-rest-1.0#commonly-accessed-resources)は、ドキュメントIDを取得する際に役立つエンドポイントのリストを提供しています。

> Another possibility is to provide a list of `object_id` for each document you want to load. For that, you will need to query the [Microsoft Graph API](https://developer.microsoft.com/en-us/graph/graph-explorer) to find all the documents ID that you are interested in. This [link](https://learn.microsoft.com/en-us/graph/api/resources/onedrive?view=graph-rest-1.0#commonly-accessed-resources) provides a list of endpoints that will be helpful to retrieve the documents ID.

例えば、`data/finance/` フォルダに格納されているすべてのオブジェクトに関する情報を取得するには、`https://graph.microsoft.com/v1.0/drives/<document-library-id>/root:/data/finance:/children` へリクエストを送る必要があります。興味のある ID のリストを手に入れたら、次に示すパラメータを用いてローダーをインスタンス化することができます。

> For instance, to retrieve information about all objects that are stored at `data/finance/` folder, you need make a request to: `https://graph.microsoft.com/v1.0/drives/<document-library-id>/root:/data/finance:/children`. Once you have the list of IDs that you are interested in, then you can instantiate the loader with the following parameters.

```python
from langchain.document_loaders.sharepoint import SharePointLoader

loader = SharePointLoader(document_library_id="YOUR DOCUMENT LIBRARY ID", object_ids=["ID_1", "ID_2"], auth_with_token=True)
documents = loader.load()
```


